In [ ]:
# Install the Earth Engine API
!pip install earthengine-api folium

# Import the Earth Engine library
import ee

# Authenticate and initialize Earth Engine
# This will open a pop-up window for you to log in with your Google account.
# Ensure it's the same account you used to sign up for GEE.
ee.Authenticate()
ee.Initialize(project='irrigation-mapping')

print("Earth Engine initialized successfully in Colab!")

Earth Engine initialized successfully in Colab!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("Google Drive mounted successfully!")
# You can now access files in '/content/drive/My Drive/'

Mounted at /content/drive
Google Drive mounted successfully!


In [ ]:
import ee
import folium
# 2. Load your exported AOI asset
# IMPORTANT: Replace 'users/YOUR_GEE_USERNAME/Polonnaruwa_AOI_Project'
# with the EXACT Asset ID you copied from the "Get Link" of your asset in the GEE Assets tab.
# It should look something like 'users/your.email@example.com/Polonnaruwa_AOI_Project'
try:
    polonnaruwa_aoi = ee.FeatureCollection('users/s19451/Polonnaruwa_AOI_Project4').first().geometry()
    print("Polonnaruwa AOI loaded successfully from GEE Assets.")
except ee.EEException as e:
    print(f"Error loading AOI asset: {e}")
    print("Please double-check your Asset ID and ensure the asset exists in your GEE Assets tab.")
    exit()

# Optional: Visualize the AOI to confirm
# Get centroid coordinates for centering the map
aoi_center = polonnaruwa_aoi.centroid().coordinates().getInfo()
map_polonnaruwa = folium.Map(location=[aoi_center[1], aoi_center[0]], zoom_start=11)

aoi_geojson = polonnaruwa_aoi.getInfo()

# Add the AOI to the Folium map using folium.GeoJson
folium.GeoJson(
    aoi_geojson,
    name='Polonnaruwa AOI',
    style_function=lambda x: {'fillColor': '#00000000', 'color': 'red'} # Transparent fill, red border
).add_to(map_polonnaruwa)


folium.LayerControl().add_to(map_polonnaruwa) # Add layer control for toggling
map_polonnaruwa # Display the map in Colab

Polonnaruwa AOI loaded successfully from GEE Assets.


In [ ]:
import ee
import folium

# Assume ee.Authenticate() and ee.Initialize() have been run
# Assume polonnaruwa_aoi is loaded correctly (e.g., from your GEE Asset)

# --- FUNCTIONS FOR IMAGE PROCESSING (keep these defined) ---
def mask_s2_clouds(image):
    """Masks clouds and shadows from a Sentinel-2 SR image."""
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
           qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000)

def add_indices(image):
    """Calculates MNDWI and NDVI and adds them as new bands."""
    mndwi = image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(mndwi).addBands(ndvi)


# --- MAIN IMAGERY PROCESSING ---

# --- DEBUGGING STEP 0: Verify AOI is loaded and not empty ---
try:
    aoi_info = polonnaruwa_aoi.getInfo()
    print(f"Polonnaruwa AOI loaded: {aoi_info['type']} with coordinates length: {len(aoi_info['coordinates'][0][0]) if aoi_info['type'] == 'Polygon' else 'N/A'}")
except Exception as e:
    print(f"ERROR: Could not retrieve info from polonnaruwa_aoi. It might be empty or invalid: {e}")
    exit()


print("Filtering Sentinel-2 harmonized imagery with maximum lenient criteria...")
# *** CRITICAL CHANGE: Switched to the harmonized dataset S2_SR_HARMONIZED ***
# *** CRITICAL CHANGE: Increased cloud cover threshold to 90% ***
s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                     .filterBounds(polonnaruwa_aoi) \
                     .filterDate('2015-01-01', '2025-06-02') \
                     .filter(ee.Filter.lt('CLOUD_COVER', 90)) \
                     .map(mask_s2_clouds) \
                     .map(add_indices)

# --- DEBUGGING STEP 1: Check how many images are in the collection ---
number_of_images = s2_collection.size().getInfo()
print(f"Number of Sentinel-2 images in collection after filtering: {number_of_images}")

if number_of_images == 0:
    print("ULTIMATE WARNING: Still no Sentinel-2 images found even with extremely lenient criteria (90% cloud, 2015-present).")
    print("This is highly unusual. Please consider the following:")
    print("1. **Double-check your AOI coordinates/location in the GEE Code Editor:** Does it actually cover a landmass in Sri Lanka, or is it mistakenly in the ocean or a very remote area? You can use `Map.addLayer(polonnaruwa_aoi)` in GEE Code Editor to confirm its exact position.")
    print("2. **Try a very small, simple test AOI:** Draw a tiny square polygon over a known land area in Sri Lanka (e.g., Colombo) and try to filter for images just for that. If that works, it indicates a problem specific to your Polonnaruwa AOI's exact boundaries or data availability for that particular large region.")
    exit() # Exit if no images are found, as further steps will fail

s2_composite = s2_collection.median().clip(polonnaruwa_aoi)
print("Sentinel-2 composite created and clipped to AOI.")

# --- DEBUGGING STEP 2: Check the bands of the composite image ---
try:
    bands_info = s2_composite.bandNames().getInfo()
    print(f"Bands available in s2_composite: {bands_info}")
except ee.EEException as e:
    print(f"ERROR: Could not retrieve bands from s2_composite. It might be empty or invalid: {e}")

# --- Visualization Code (will run if images found and bands are present) ---
required_bands = ['B4', 'B3', 'B2', 'MNDWI']
if number_of_images > 0 and all(band in bands_info for band in required_bands):
    vis_params_s2 = {'bands': ['B4', 'B3', 'B2'], 'min': 0.0, 'max': 0.3}
    vis_params_mndwi = {'min': -0.5, 'max': 0.5, 'palette': ['red', 'white', 'blue']}

    aoi_center = polonnaruwa_aoi.centroid().coordinates().getInfo()
    map_polonnaruwa = folium.Map(location=[aoi_center[1], aoi_center[0]], zoom_start=11)

    folium.GeoJson(
        polonnaruwa_aoi.getInfo(),
        name='Polonnaruwa AOI',
        style_function=lambda x: {'fillColor': '#00000000', 'color': 'red'}
    ).add_to(map_polonnaruwa)

    folium.TileLayer(
        tiles=s2_composite.getMapId(vis_params_s2)['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        overlay=True,
        name='Sentinel-2 Composite (Natural Color)'
    ).add_to(map_polonnaruwa)

    folium.TileLayer(
        tiles=s2_composite.select('MNDWI').getMapId(vis_params_mndwi)['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        overlay=True,
        name='MNDWI (Water Index)'
    ).add_to(map_polonnaruwa)

    folium.LayerControl().add_to(map_polonnaruwa)
    map_polonnaruwa
else:
    print("Skipping map visualization because no images found or required bands are missing after composite creation.")

Polonnaruwa AOI loaded: Polygon with coordinates length: 2
Filtering Sentinel-2 harmonized imagery with maximum lenient criteria...
Number of Sentinel-2 images in collection after filtering: 0
ULTIMATE WARNING: Still no Sentinel-2 images found even with extremely lenient criteria (90% cloud, 2015-present).
This is highly unusual. Please consider the following:
1. **Double-check your AOI coordinates/location in the GEE Code Editor:** Does it actually cover a landmass in Sri Lanka, or is it mistakenly in the ocean or a very remote area? You can use `Map.addLayer(polonnaruwa_aoi)` in GEE Code Editor to confirm its exact position.
2. **Try a very small, simple test AOI:** Draw a tiny square polygon over a known land area in Sri Lanka (e.g., Colombo) and try to filter for images just for that. If that works, it indicates a problem specific to your Polonnaruwa AOI's exact boundaries or data availability for that particular large region.
Sentinel-2 composite created and clipped to AOI.
Bands 

In [ ]:
import ee
import folium

# Assuming ee.Authenticate() and ee.Initialize() have been run successfully
# If not, please run them first:
try:
    ee.Authenticate()
    ee.Initialize(project='irrigation-mapping')
    print("Earth Engine initialized successfully!")
except ee.EEException as e:
    print(f"Error initializing Earth Engine: {e}")
    exit()

# Define a small test point near Kandy, Sri Lanka (your current location)
test_point = ee.Geometry.Point(80.635, 7.291) # Coordinates for Kandy

print("Attempting to filter Sentinel-2 for a test point in Sri Lanka...")
test_s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                         .filterBounds(test_point) \
                         .filterDate('2023-01-01', '2024-01-01') \
                         .filter(ee.Filter.lt('CLOUD_COVER', 50)) # Using a more moderate cloud filter for this test

test_image_count = test_s2_collection.size().getInfo()
print(f"Number of Sentinel-2 images found for test point: {test_image_count}")

if test_image_count == 0:
    print("CRITICAL: Still no images found even for a small test point. This is highly unusual.")
    print("Please ensure your Earth Engine authentication is still valid and try restarting your Colab runtime.")
else:
    print("Images found for test point. The issue seems specific to your Polonnaruwa AOI's exact shape/location or extreme data characteristics.")

# Optional: Visualize a test composite if images are found
if test_image_count > 0:
    test_composite = test_s2_collection.median().clip(test_point.buffer(500)) # Buffer point for visibility
    vis_params_test = {'bands': ['B4', 'B3', 'B2'], 'min': 0.0, 'max': 0.3}

    map_test = folium.Map(location=[test_point.coordinates().getInfo()[1], test_point.coordinates().getInfo()[0]], zoom_start=12)
    folium.TileLayer(
        tiles=test_composite.getMapId(vis_params_test)['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        overlay=True,
        name='Test Sentinel-2 Composite'
    ).add_to(map_test)
    folium.GeoJson(
        test_point.getInfo(),
        name='Test Point',
        style_function=lambda x: {'fillColor': 'blue', 'color': 'blue'}
    ).add_to(map_test)
    folium.LayerControl().add_to(map_test)
    map_test

Earth Engine initialized successfully!
Attempting to filter Sentinel-2 for a test point in Sri Lanka...
Number of Sentinel-2 images found for test point: 0
CRITICAL: Still no images found even for a small test point. This is highly unusual.
Please ensure your Earth Engine authentication is still valid and try restarting your Colab runtime.
